
![alt text](https://i.imgur.com/HRhd2Y0.png)

In [5]:
import pandas as pd

from functions import find_csv_files, load_csvs_to_dict, sort_and_classify_column

DATA_PATH = "/Users/typhaine/Documents/Doc_Gorilla/OpenClassroom--Machine-Learning-Engineer/P4/data"

csv_files = find_csv_files(DATA_PATH)

dfs = load_csvs_to_dict(csv_files)


In [6]:
recency_df = sort_and_classify_column(dfs["olist_orders_dataset"], column_name = "order_purchase_timestamp", datetime = True)
recency_df

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_purchase_timestamp_class
4541,2e7a8482f6fb09756ca50c10d7bfc047,08c5351a6aca1c1589a38f244edeee9d,shipped,2016-09-04 21:15:19,2016-10-07 13:18:03,2016-10-18 13:14:51,NaN,2016-10-20 00:00:00,Low
4396,e5fa5a7210941f7d56d0208e4e071d35,683c54fc24d40ee9f8a6fc179fd9856c,canceled,2016-09-05 00:15:34,2016-10-07 13:17:15,NaN,NaN,2016-10-28 00:00:00,Low
10071,809a282bbd5dbcabb6f2f724fca862ec,622e13439d6b5a0b486c435618b2679e,canceled,2016-09-13 15:24:19,2016-10-07 13:16:46,NaN,NaN,2016-09-30 00:00:00,Low
30710,bfbd0f9bdef84302105ad712db648a6c,86dc2ffce2dfff336de2f386a786e574,delivered,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-11-07 17:11:53,2016-11-09 07:47:38,2016-10-04 00:00:00,Low
83078,71303d7e93b399f5bcd537d124c0bcfa,b106b360fe2ef8849fbbd056f777b4d5,canceled,2016-10-02 22:07:52,2016-10-06 15:50:56,NaN,NaN,2016-10-25 00:00:00,Low
...,...,...,...,...,...,...,...,...,...
50387,392ed9afd714e3c74767d0c4d3e3f477,2823ffda607a2316375088e0d00005ec,canceled,2018-09-29 09:13:03,NaN,NaN,NaN,2018-10-15 00:00:00,High
88500,616fa7d4871b87832197b2a137a115d2,bf6181a85bbb4115736c0a8db1a53be3,canceled,2018-10-01 15:30:09,NaN,NaN,NaN,2018-10-23 00:00:00,High
31891,a2ac6dad85cf8af5b0afb510a240fe8c,4c2ec60c29d10c34bd49cb88aa85cfc4,canceled,2018-10-03 18:55:29,NaN,NaN,NaN,2018-10-16 00:00:00,High
68373,b059ee4de278302d550a3035c4cdb740,856336203359aa6a61bf3826f7d84c49,canceled,2018-10-16 20:16:02,NaN,NaN,NaN,2018-11-12 00:00:00,High


In [7]:
frequency_df = dfs["olist_orders_dataset"].merge(dfs["olist_customers_dataset"], on = "customer_id").groupby(by = "customer_unique_id").count().reset_index().loc[:, ["customer_unique_id" ,"order_id"]]
frequency_df = sort_and_classify_column(frequency_df, column_name = "order_id")
frequency_df

,customer_unique_id,order_id,order_id_class
0,0000366f3b9a7992bf8c76cfdf3221e2,1,Low
63580,a926cfc9bc7b082335de50450f48eec9,1,Low
63579,a926aae38267e7f54e67de9b5775d0a5,1,Low
63578,a92605ec492805540520d3a73aaeeb6e,1,Low
63577,a925c3e5df82fdc6082f1383d2834998,1,Low
...,...,...,...
76082,ca77025e7201e3b30c44b472ff346268,7,Medium
37797,6469f99c1f9dfae7733b25662e7f1782,7,Medium
10354,1b6c7548a2a1f9037c1fd3ddfed95f33,7,Medium
23472,3e43e6105506432c953e165fb2acf44c,9,Medium


In [8]:
per_order_payment = dfs["olist_order_payments_dataset"].groupby(by = "order_id").sum().reset_index()
monetary_df = dfs["olist_orders_dataset"].merge(per_order_payment, on = "order_id")
monetary_df = sort_and_classify_column(monetary_df, column_name = "payment_value")
monetary_df

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,payment_installments,payment_value,payment_value_class
40234,c8c528189310eaa44a745b8d9d26908b,197a2a6a77da93f678ea0d379f21da0a,canceled,2018-08-28 20:05:14,NaN,NaN,NaN,2018-09-11 00:00:00,1,not_defined,1,0.00,Low
39918,4637ca194b6387e2d538dc89b124b0ee,a73c1f73f5772cf801434bf984b0b1a7,canceled,2018-09-03 14:14:25,NaN,NaN,NaN,2018-09-10 00:00:00,1,not_defined,1,0.00,Low
1130,00b1cb0320190ca0daa2c88b35206009,3532ba38a3fd242259a514ac2b6ae6b6,canceled,2018-08-28 15:26:39,NaN,NaN,NaN,2018-09-12 00:00:00,1,not_defined,1,0.00,Low
95517,f1d5c2e6867fa93ceee9ef9b34a53cbf,a790343ca6f3fee08112d678b43aa7c5,delivered,2018-08-25 21:20:50,2018-08-25 21:30:15,2018-08-28 10:49:00,2018-08-29 18:19:41,2018-09-03 00:00:00,3,credit_cardvoucher,2,9.59,Low
11601,e8bbc1d69fee39eee4c72cb5c969e39d,184e8e8e48937145eb96c721ef1f0747,delivered,2017-09-13 19:13:20,2017-09-13 19:25:38,2017-09-15 17:43:43,2017-09-18 18:42:07,2017-09-26 00:00:00,1,credit_card,1,10.07,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3508,f5136e38d1a14a4dbd87dff67da82701,3fd6777bbce08a352fddd04e4a7cc8f6,delivered,2017-05-24 18:14:34,2017-05-26 02:45:17,2017-05-26 11:20:47,2017-06-05 17:09:48,2017-06-28 00:00:00,1,boleto,1,6726.66,Medium
28326,fefacc66af859508bf1a7934eab1e97f,f48d464a0baaea338cb25f816991ab1f,delivered,2018-07-25 18:10:17,2018-07-27 04:05:13,2018-08-03 14:42:00,2018-08-15 14:57:50,2018-08-10 00:00:00,1,boleto,1,6922.21,Medium
22171,0812eb902a67711a1cb742b3cdaa65ae,c6e2731c5b391845f6800c97401a43a9,delivered,2017-02-12 20:37:36,2017-02-12 20:45:12,2017-02-16 09:23:13,2017-03-03 14:23:18,2017-03-09 00:00:00,1,credit_card,8,6929.31,Medium
66598,736e1922ae60d0d6a89247b851902527,ec5b2ba62e574342386871631fafd3fc,delivered,2018-07-15 14:49:44,2018-07-17 04:31:36,2018-07-20 13:09:00,2018-07-26 22:03:06,2018-08-02 00:00:00,1,boleto,1,7274.88,Medium


In [11]:
#recency_df.merge()
dfs["olist_customers_dataset"].merge(frequency_df, on = "customer_unique_id")

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,order_id_class
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,1,Low
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,1,Low
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,1,Low
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,1,Low
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,1,Low
...,...,...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP,1,Low
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP,1,Low
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE,1,Low
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS,1,Low
